In [348]:
# feature extraction and data preprocessing
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv
import numpy as np
import IPython.display as ipd

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras

import warnings
warnings.filterwarnings('ignore')

# Acquire and Prepare Dataset
- Read in audio features from the GTZAN dataset. (download from: https://www.kaggle.com/andradaolteanu/gtzan-dataset-music-genre-classification)

In [349]:
data = pd.read_csv('DATA/GTZAN/features_30_sec.csv')
data.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.wav,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,...,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,blues.00001.wav,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,...,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,blues.00002.wav,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,...,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,blues.00003.wav,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,...,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,blues.00004.wav,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,...,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues


In [350]:
data.shape

(1000, 60)

In [351]:
# Dropping unneccesary columns
genre_list = data.iloc[:, -1]
data = data.drop(['filename', 'length',  'harmony_mean', 'harmony_var', 'perceptr_mean', 'perceptr_var', 'tempo'],axis=1)
data = data.drop(['label'],axis=1)
data.head()

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
0,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,3805.839606,9.015054e+05,...,0.752740,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035
1,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,3550.522098,2.977893e+06,...,0.927998,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282
2,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,3042.260232,7.840345e+05,...,2.451690,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025
3,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,2184.745799,1.493194e+06,...,0.780874,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339
4,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,3579.757627,1.572978e+06,...,-4.520576,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160


### Get the genre list. We'll use it later to calculate the error

In [352]:
genres_ordered = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

In [353]:
scaler = StandardScaler()
X_arr = np.array(data.iloc[:, :-1], dtype = float)
X = scaler.fit_transform(X_arr)
X

array([[-0.35013678,  0.31258717, -0.01068969, ...,  0.00672291,
        -0.30059734,  0.60406407],
       [-0.46248155,  1.11757233, -0.53285232, ...,  0.54480563,
        -0.40708699,  0.42412706],
       [-0.18422456, -0.13770124,  0.67997762, ..., -0.29593404,
        -0.52729705, -0.29618888],
       ...,
       [ 0.65463736, -1.43198917, -0.75062494, ..., -2.73284378,
        -0.63865065, -0.26361549],
       [-0.19833855,  0.66814351, -0.71697762, ..., -0.72271696,
        -0.5114848 , -0.65064889],
       [-0.2483391 , -0.05894495, -1.1648952 , ...,  0.08070645,
         0.16033426,  0.5868411 ]])

# Setup Training vs Testing Data
- Dividing data into training and Testing set

In [354]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [355]:
len(y_train)

800

In [356]:
len(y_test)

200

In [357]:
X_train[10]

array([ 0.45415235, -0.96333601,  0.65481871, -0.21442785,  0.19656154,
       -0.53141412, -0.24617751, -0.67735263,  0.01515206, -0.57699313,
        0.83327186, -0.25386623,  1.09671953, -0.92857794,  0.22162514,
       -0.49969816, -1.48256695, -0.27885264, -0.15771986, -0.8324527 ,
       -2.13549948, -0.76598242,  0.57224818, -0.54446945, -1.62873761,
       -0.12035316,  0.37244698, -1.07117442, -2.50973435,  0.54443445,
        1.0492596 , -0.08466683, -1.87797681, -0.79603894,  0.17462611,
       -0.70303736, -1.54041795, -0.67077808, -0.19485562, -0.41278848,
       -1.3685084 , -0.62180584,  0.71941436, -0.21872174, -0.7692418 ,
       -0.67695005,  0.4041564 , -0.17381702, -0.1515229 , -0.68432441,
        0.19088737])

# Setup Network Arhitecture
- We setup a feed forward deep learning neural network.
- It has 5 densely connected layers. The 1st 4 layers have relu activation, and the final layer is softmax.
- We have one layer for each genre

In [358]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(256, activation='relu'))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))  # Last layer, finds the most probable genre

# Define Network Loss and Optimization

In [359]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the network
- We set a batch size of 256 and train for 80 epochs

In [360]:
history = model.fit(X_train,
                    y_train,
                    epochs=80,
                    batch_size=256)

Epoch 1/80
4/4 [==============================] - 0s 4ms/step - loss: 2.2560 - accuracy: 0.1704
Epoch 2/80
4/4 [==============================] - 0s 4ms/step - loss: 1.9086 - accuracy: 0.3969
Epoch 3/80
4/4 [==============================] - 0s 4ms/step - loss: 1.5957 - accuracy: 0.4978
Epoch 4/80
4/4 [==============================] - 0s 3ms/step - loss: 1.3299 - accuracy: 0.5965
Epoch 5/80
4/4 [==============================] - 0s 3ms/step - loss: 1.1085 - accuracy: 0.6494
Epoch 6/80
4/4 [==============================] - 0s 4ms/step - loss: 0.9412 - accuracy: 0.6991

# Evaluate the model with our testing data

In [361]:
test_loss, test_acc = model.evaluate(X_test,y_test)

7/7 [==============================] - 0s 1ms/step - loss: 1.4785 - accuracy: 0.7250


In [362]:
print('test accuracy: ',test_acc)

test accuracy:  0.7250000238418579


# Model Inference
- We can now use our model to make predictions for other songs. First, we need to extract audio features from those songs, and put them in a .csv

In [363]:
def create_csv(track_name, csv_name):
    input_freq_data, sr = librosa.load('./audio/'+track_name, mono=True, duration=30)


    chroma_stft = librosa.feature.chroma_stft(y=input_freq_data, sr=sr)
    rmse = librosa.feature.rms(y=input_freq_data)
    spec_cent = librosa.feature.spectral_centroid(y=input_freq_data, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=input_freq_data, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=input_freq_data, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(input_freq_data)
    mfcc = librosa.feature.mfcc(y=input_freq_data, sr=sr)


    to_append = f'{track_name[:-4]} {np.mean(chroma_stft)} {np.var(chroma_stft)} {np.mean(rmse)} {np.var(rmse)} {np.mean(spec_cent)} {np.var(spec_cent)} {np.mean(spec_bw)} {np.var(spec_bw)} {np.mean(rolloff)} {np.var(rolloff)} {np.mean(zcr)} {np.var(zcr)}'
    for e in mfcc:
        to_append += f' {np.mean(e)}'
        to_append += f' {np.var(e)}'

    file = open(csv_name, 'a', newline='')
    with file:
        writer = csv.writer(file)
        writer.writerow(to_append.split())

def create_header(csv_name):
    header = 'filename chroma_stft_mean chroma_stft_var rms_mean rms_var spectral_centroid_mean spectral_centroid_var spectral_bandwidth_mean spectral_bandwidth_var rolloff_mean rolloff_var zero_crossing_rate_mean zero_crossing_rate_var'
    for i in range(1, 21):
        header += f' mfcc{i}_mean'
        header += f' mfcc{i}_var'
    header += ' label'
    header = header.split()

    file = open(csv_name, 'w', newline='')
    with file:
        writer = csv.writer(file)
        writer.writerow(header)


In [ ]:
csv_name = 'data.csv'
tracks = os.listdir('./audio')
# track_name = tracks[2]
print(tracks)

# Create CSV
create_header(csv_name)
for track in tracks:
    create_csv(track, csv_name)

['stayin_alive_the_begees.wav', 'cliffs_of_dover_eric_johnson.wav', 'forward.wav', 'gangplank_galleon.wav', 'accoustic_guitar.wav', 'moonlight_sonata.wav', '1812_overture.wav', 'subnautica_below_zero.wav', 'bluegrass_riff.wav', 'in_a_sentimental_mood.wav']


### Now let's read our input data.

In [ ]:
# now let's
in_data = None
for track in tracks:
    in_data = pd.read_csv(csv_name)
    in_data = in_data.drop(['filename', 'label'],axis=1)

In [ ]:
print("Here's what our input data looks like")
in_data.head()

In [ ]:
in_arr = np.array(in_data.iloc[:, :-1], dtype = float)
scaler = StandardScaler()
arr = scaler.fit_transform(in_arr)  # THIS NEEDS TO HAVE MORE THAN 1 SONG----- WHAT

# arr

In [ ]:
predictions = model.predict(arr)

In [ ]:
print("Here's the prediction strengths of each genre for song 1")
predictions[0]

In [ ]:
np.sum(predictions[0]) # checks that add to ~1

In [ ]:
for i in range(len(tracks)):
    print("The predicted genre for " + str(tracks[i]) + " is " +str(genres_ordered[ np.argmax(predictions[i]) ])+ "!\n")